In [ ]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/학교수업/캡스톤/data/'

In [ ]:
with open(path+"train_id_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open(path+"valid_id_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open(path+"test_id_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [ ]:
with open(path+"train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open(path+"valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open(path+"test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [ ]:
# ID
train_data[0] = tf.convert_to_tensor(train_data[0], dtype=tf.float32)
train_data[1] = tf.convert_to_tensor(train_data[1], dtype=tf.float32)

## Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add, GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 5
#current_path ='data/'
#local_path = 'data/'

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
# set parameters:
# V = 142381
# V = 226928
V = 48456 #  33531 6158 48456
embedding_dim = 50
max_len = 1 #  1945 77 1
filters_num = 64
kernel_size = 3

head_size = 10
num_heads = 5  # Number of attention heads
ff_dim = 4  # Hidden layer size in feed forward network inside transformer
dropout = 0.2

In [ ]:
# # description_input=Input(shape=(10,50))
# # description_validation=Input(shape=(50,))
# input_1 = Input(shape=(description_num, max_len))
# input_2 = Input(shape=(max_len,))
# embedding = Embedding(input_dim=V,
#                       output_dim=embedding_dim,
#                       input_length=max_len)
# conv1d = Conv1D(filters=filters_num,
#                 kernel_size=kernel_size,
#                 activation='relu')
# maxpool1d = MaxPool1D(max_len-2)
# dense = Dense(15)


# # transformer
# layer_norm = layers.LayerNormalization(epsilon=1e-6)
# MHA = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)
# dropout = layers.Dropout(dropout)

# # input_vector=TimeDistributed(embedding)(description_input)
# # validation_vector=embedding(description_validation)

# input_vector = TimeDistributed(embedding)(input_1)
# validation_vector = embedding(input_2)

# te_vector = transformer_encoder(input_vector, head_size, num_heads, ff_dim)
# validation_te = transformer_encoder(validation_vector, head_size, num_heads, ff_dim)

# maxpooling_vector = TimeDistributed(maxpool1d)(te_vector)
# validation_maxpooling = maxpool1d(validation_te)

# middle_output = TimeDistributed(dense)(maxpooling_vector)
# middle_validation = dense(validation_maxpooling)

In [ ]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))
input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu')
maxpool1d = MaxPool1D(max_len)
dense = Dense(15)

# transformer
layer_norm = layers.LayerNormalization(epsilon=1e-6)
MHA = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=0.2)
val_MHA = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=0.2)
dropout = layers.Dropout(0.2)
conv1d_f = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")
conv1d_f_50 = Conv1D(filters=50 , kernel_size=1)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

input_vector = TimeDistributed(layer_norm)(input_vector)
validation_vector = layer_norm(validation_vector)

mha_vector = MHA(input_vector, input_vector)
validation_mha  = val_MHA(validation_vector ,validation_vector)

mha_vector = TimeDistributed(dropout)(mha_vector)
validation_mha = dropout(validation_mha)

add_vector = Add()([input_vector, mha_vector ])
validation_add = Add()([validation_mha , validation_mha ])

ff_vector = TimeDistributed(layer_norm)(add_vector)
validation_ff = layer_norm(validation_add)

ff_vector = TimeDistributed(conv1d_f)(ff_vector)
validation_ff = conv1d_f(validation_ff)

ff_vector = TimeDistributed(dropout)(ff_vector)
validation_ff = dropout(validation_ff)

ff_vector = TimeDistributed(conv1d_f_50)(ff_vector)
validation_ff = conv1d_f_50(validation_ff)

ff_vector = Add()([add_vector, ff_vector ])
validation_ff = Add()([validation_add , validation_ff ])

maxpooling_vector = TimeDistributed(maxpool1d)(ff_vector)
validation_maxpooling = maxpool1d(validation_ff)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [ ]:
print(input_vector.shape, validation_vector.shape,
#convolutional_vector.shape, validation_conv.shape,
maxpooling_vector.shape, validation_maxpooling.shape,
middle_output.shape, middle_validation.shape)

(None, 5, 1, 50) (None, 1, 50) (None, 5, 1, 50) (None, 1, 50) (None, 5, 1, 15) (None, 1, 15)


In [ ]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [ ]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [ ]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', 'mape'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 1)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 5, 1, 50)    2422800     ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        2422800     ['input_2[0][0]']            

In [ ]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 18s 15ms/step - loss: 18.9796 - mse: 18.9796 - val_loss: 11.4269 - val_mse: 11.4269
Epoch 2/30
256/256 [==============================] - 2s 9ms/step - loss: 11.0916 - mse: 11.0916
Epoch 3/30
256/256 [==============================] - 2s 9ms/step - loss: 4.7381 - mse: 4.7381
Epoch 4/30
256/256 [==============================] - 2s 9ms/step - loss: 3.0252 - mse: 3.0252
Epoch 5/30
256/256 [==============================] - 2s 9ms/step - loss: 2.5853 - mse: 2.5853
Epoch 6/30
256/256 [==============================] - 2s 9ms/step - loss: 2.3795 - mse: 2.3795
Epoch 7/30
256/256 [==============================] - 2s 9ms/step - loss: 2.2400 - mse: 2.2400
Epoch 8/30
256/256 [==============================] - 2s 9ms/step - loss: 2.1383 - mse: 2.1383
Epoch 9/30
256/256 [==============================] - 2s 9ms/step - loss: 2.0444 - mse: 2.0444
Epoch 10/30
256/256 [==============================] - 2s 9ms/step - loss: 1.9718 - mse: 1.9718
Epoc

In [ ]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 9s 4ms/step - loss: 2.9177 - mse: 2.9177
[2.91770601272583, 2.91770601272583]


In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/학교수업/캡스톤/model/Anime_30_name_TE")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/학교수업/캡스톤/model/Anime_30_name_TE/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/학교수업/캡스톤/model/Anime_30_name_TE/assets
